In [1]:
# import other
%run other.ipynb

In [1]:
import numpy as np
import time
import math
import copy
from PIL import Image, ImageDraw

In [4]:
class filter_class:

    def gaussian_filter_padding(image_array, Variance_mode ='gaussian',n=3):
        # Gaussian所需要的參數
        gauss_value = np.zeros(3)
        Variance_1 = np.zeros(3)
        Variance = np.zeros(3)
        average = np.zeros(3)
        num = n**2

        
        h1, w1, r1 = image_array.shape
        padding_array_h = image_array[0,:]
        padding = int(n//2)
        for i in range(padding):
            image_array = np.insert(image_array, 0, padding_array_h, axis=0) #上
            image_array = np.insert(image_array, h1, padding_array_h, axis=0) #下
        padding_array_w = image_array[:,0]
        for i in range(padding):
            image_array = np.insert(image_array, 0, padding_array_w, axis=1) #左
            image_array = np.insert(image_array, w1, padding_array_w, axis=1) #右

        image_array_list = []

        mesh = np.zeros((n,n,(3)), dtype='uint8')
        h, w, z = image_array.shape #5 5 3
        for y in np.arange(0, h, 1):
            for x in np.arange(0, w, 1):
                if   (x+n) < w and (y+n) < h:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(1)
                elif (x+n) == w and (y+n) < h:
                    mesh = image_array[y:y+n, w-n:w]
                    #print(2)
                elif (x+n) < w and (y+n) == h:
                    mesh = image_array[h-n:h, x:x+n]
                    #print(3)
                elif (x+n) == w and (y+n) == h:
                    mesh = image_array[h-n:h, w-n:w]
                    #print(4)
                else:
                    break
                    print("X")
                #print(mesh.shape)
                # 計算Variance
                if Variance_mode == 'gaussian': 
                    Variance_1 = np.zeros(3)
                    Variance = np.zeros(3)
                    for y2 in np.arange(0, n, 1):
                        for x2 in np.arange(0, n, 1):
                            for rgb in range(3):
                                average[rgb] = np.mean(mesh[:,:,(rgb)])
                                Variance_1[rgb] = (mesh[y2,x2,(rgb)]-average[rgb])**2
                                Variance[rgb] += Variance_1[rgb]
                    for rgb in range(3):
                        Variance[rgb] = Variance[rgb] / num
                        if Variance[rgb] ==0 or Variance[rgb] < 0.001:
                            Variance[rgb] = 0.01
                else:
                    Variance = np.array([Variance_mode, Variance_mode, Variance_mode])

                gaussian_mesh = np.zeros((n,n,(3)), dtype='float')

                for y1 in np.arange(padding+1, -padding-2, -1):
                    for x1 in np.arange(-padding-1, padding+2, 1):
                        for rgb in range(3):
                            #print(f'{x1},{y1}')
                            gaussian_mesh[y1, x1, (rgb)] = (1/(2*(math.pi)*Variance[rgb]))*(math.exp(-(x1**2+y1**2)/(2*Variance[rgb])))
                            #print(gaussian_mesh[x1, y1, (rgb)])
                #print('*',gaussian_mesh.shape)
                for rgb in range(3):
                    sum_gaussian = np.sum(gaussian_mesh[:,:,(rgb)])
                    if sum_gaussian < 0.000001:
                        sum_gaussian = 0.01
                    gaussian_mesh[:,:,(rgb)] = gaussian_mesh[:,:,(rgb)] / sum_gaussian

                    #mesh[padding+1, padding+1,(rgb)] = np.sum(mesh[:,:,(rgb)]*gaussian_mesh[:,:, (rgb)])
                    image_array_list.append(np.sum(mesh[:,:,(rgb)]*gaussian_mesh[:,:, (rgb)]))
                    #print(a[rgb])
        image_array_new = np.array(image_array_list, dtype='uint8')
        image_array_new = image_array_new.reshape([h1, w1 ,(r1)])
        return image_array_new
    
    def gaussian_filter_mesh_mesh(image_array_mesh_input, image, Variance_mode ='gaussian',n=3):
        import math
        image_array_mesh = copy.copy(image_array_mesh_input)
        
        # Gaussian所需要的參數
        gauss_value = np.zeros(3)
        Variance_1 = np.zeros(3)
        Variance = np.zeros(3)
        average = np.zeros(3)
        num = n**2

        # 將大mesh逐一計算
        x_scale, y_scale = other_class.mesh_search(image_array_mesh, image)
        for x in range(x_scale):
            for y in range(y_scale):
                # Gaussian mesh
                mesh_temp = image_array_mesh[y, x]
                mesh = np.zeros((n,n,(3)), dtype='uint8')
                x_mesh, y_mesh, z = mesh_temp.shape #5 5 3

                #-----------------------------------計算Variance------------------------------------
                # 對一個大mesh做 卷積，逐一取出小 mesh
                for y2 in np.arange(0, y_mesh, 1):
                    for x2 in np.arange(0, x_mesh, 1):
                        if   (x2+n) < x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x2:x2+n, y2:y2+n]
                            #print(1)
                        elif (x2+n) == x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y2:y2+n]
                            #print(2)
                        elif (x2+n) < x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x2:x2+n, y_mesh-n:y_mesh]
                            #print(3)
                        elif (x2+n) == x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y_mesh-n:y_mesh]
                            #print(4)
                        else:
                            break
                            print("X")
                # 計算Variance
                if Variance_mode == 'gaussian': 
                    Variance_1 = np.zeros(3)
                    Variance = np.zeros(3)
                    for y2 in np.arange(0, n, 1):
                        for x2 in np.arange(0, n, 1):
                            for rgb in range(3):
                                average[rgb] = np.mean(mesh[:,:,(rgb)])
                                Variance_1[rgb] = (mesh[x2,y2,(rgb)]-average[rgb])**2
                                Variance[rgb] += Variance_1[rgb]
                    for rgb in range(3):
                        Variance[rgb] = Variance[rgb] / num
                        if Variance[rgb] ==0 or Variance[rgb] < 0.001:
                            Variance[rgb] = 0.01
                else:
                    Variance = np.array([Variance_mode, Variance_mode, Variance_mode])
                    
                #------------將各mesh的Variance逐一計算高斯權重並且乘上mesh內的RGB值，取代(0,0)的pixel----------------------------------
                for y2 in np.arange(0, y_mesh, 1):
                    for x2 in np.arange(0, x_mesh, 1):
                        if   (x2+n) < x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x2:x2+n, y2:y2+n]
                            #print(1)
                        elif (x2+n) == x_mesh and (y2+n) < y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y2:y2+n]
                            #print(2)
                        elif (x2+n) < x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x2:x2+n, y_mesh-n:y_mesh]
                            #print(3)
                        elif (x2+n) == x_mesh and (y2+n) == y_mesh:
                            mesh = mesh_temp[x_mesh-n:x_mesh, y_mesh-n:y_mesh]
                            #print(4)
                        else:
                            break
                            print("X")

                        gaussian_mesh = np.zeros((n,n,(3)), dtype='float')
                        count = 0
                        for cy in range(n):
                            for cx in range(n):
                                for y1 in np.arange(-cy, n-cy, 1):
                                    for x1 in np.arange(-cx, n-cx, 1):
                                        for rgb in range(3):
                                            #print(Variance[rgb])
                                            #print(x1)
                                            #print(y1)
                                            gaussian_mesh[x1, y1, (rgb)] = (1/(2*(math.pi)*Variance[rgb]))*(math.exp(-(x1**2+y1**2)/(2*Variance[rgb])))
                                            #print(gaussian_mesh[x1, y1, (rgb)])
                                #print(gaussian_mesh)
                                for rgb in range(3):
                                    sum_gaussian = np.sum(gaussian_mesh[:,:,(rgb)])
                                    if sum_gaussian < 0.000001:
                                        sum_gaussian = 0.01
                                    gaussian_mesh[:,:,(rgb)] = gaussian_mesh[:,:,(rgb)] / sum_gaussian
                                    #print(gaussian_mesh[:,:,(rgb)])
                                    mesh[cx, cy,(rgb)] = np.sum(mesh[:,:,(rgb)]*gaussian_mesh[:,:, (rgb)])
                                    #print(a[rgb])
        return image_array_mesh
  
    def Sharpening_filter_laplacian_3x3(image_array, c=1, level=32):
        '''
        這裡的卷積方法是沿著x方向
        這裡會使用邊緣擴增的方式來完成捲積
        c : 銳化遮罩的強度，數字越小，銳化強度越低
        '''
        n = 3 # kernel
        original_array = image_array
        # image.size   (233, 204)
        h, w, r = image_array.shape # (204, 233, 3)

        zero_array = np.zeros(1)
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下
        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右

        h1, w1, r1 = image_array.shape

        # 銳化的mesh
        window = np.array((
            [-1, -1, -1],
            [-1, 8, -1],
            [-1, -1, -1]), dtype="int")

        sharpen_mesh_list = []
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    pixel_value =  np.sum(mesh[rgb]*window)/level
                    if pixel_value<0:
                        pixel_value = 0
                    elif pixel_value > 255:
                        pixel_value = 255
                    else:
                        pass
                    sharpen_mesh_list.append(pixel_value)
        # 將list轉成array
        sharpen_mesh_array = np.array(sharpen_mesh_list, dtype='int')
        #對array做reshape
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        sharpen_mesh_array_reshape = original_array + c * sharpen_mesh_array_reshape
        for y in np.arange(0, h, 1):
            for x in np.arange(0, w, 1):
                for rgb in range(r):
                    pixel_value = sharpen_mesh_array_reshape[y,x,(rgb)]
                    if pixel_value < 0:
                        pixel_value = 0
                    elif pixel_value > 255:
                        pixel_value = 255
                    else:
                        pass
                    sharpen_mesh_array_reshape[y,x,(rgb)] = pixel_value
        sharpen_mesh_array_reshape = sharpen_mesh_array_reshape.astype('uint8')
        return sharpen_mesh_array_reshape
    
    '''
    # y方向的卷積
    def Sharpening_filter_laplacian_3x3(image_array):
        h, w, r = image_array.shape
        zero_array = np.zeros(n)
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下
        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右
        
        h1, w1, r1 = image_array.shape
        
        # 銳化的mesh
        window = ([
            [-1, -1, -1],
            [-1,  8, -1],
            [-1, -1, -1]
        ])
        init = 0
        temp_b = np.zeros([h,1,(3)], dtype='uint8')
        sharpen_mesh_list = []
        for x in np.arange(0, w1-1, 1):
            if init == 1:

                a = np.array(sharpen_mesh_list , dtype='uint8')
                #print(a.shape)
                b = a.reshape([h,1,(3)])
                temp_b = np.concatenate((temp_b, b), axis = 1)
                sharpen_mesh_list = []
            for y in np.arange(0, h1, 1):
                if x==w1:
                    continue
                init = 1
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    pixel_value =  np.sum(mesh[rgb]*window)/8
                    if pixel_value<0:
                        pixel_value = 0
                    elif pixel_value>255:
                        pixel_value = 255
                    else:
                        pass
                    sharpen_mesh_list.append(pixel_value)
        temp_b = np.delete(temp_b, 0, axis = 1)
        return temp_b
    '''
    
    def Sharpening_filter_laplacian_5x5(image_array, n=5):
        h, w, r = image_array.shape

        zero_array = np.zeros(1)
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下
        image_array = np.insert(image_array, 0, zero_array, axis=0) #上
        image_array = np.insert(image_array, h, zero_array, axis=0) #下

        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右
        image_array = np.insert(image_array, 0, zero_array, axis=1) #左
        image_array = np.insert(image_array, w, zero_array, axis=1) #右
        h1, w1, r1 = image_array.shape
        # 銳化的mesh 5*5
        window = ([
            [-1, -1, -1, -1, -1],
            [-1,  2,  2,  2, -1],
            [-1,  2,  8,  2, -1],
            [-1,  2,  2,  2, -1],
            [-1, -1, -1, -1, -1]
        ])
        sharpen_mesh_list = []
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh[:,:,(0)])
                    #print(y, x)
                for rgb in range(3):
                    pixel_value =  np.sum(mesh[:,:,(rgb)]*window)/8
                    if pixel_value < 0:
                        pixel_value = 0
                    elif pixel_value>255:
                        pixel_value = 255
                    else:
                        pass
                    sharpen_mesh_list.append(pixel_value)
        sharpen_mesh_array = np.array(sharpen_mesh_list, dtype='uint8')
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        return sharpen_mesh_array_reshape
    
    def median_filter(image_array, n = 3):
        '''
        中值濾波
        x方向卷積
        n最小為3 (限奇數)
        '''
        h, w, r = image_array.shape
        zero_array = np.zeros(1)
        padding = int(n//2)
        for i in range(padding):
            image_array = np.insert(image_array, 0, zero_array, axis=0) #上
            image_array = np.insert(image_array, h, zero_array, axis=0) #下
            image_array = np.insert(image_array, 0, zero_array, axis=1) #左
            image_array = np.insert(image_array, w, zero_array, axis=1) #右
        
        h1, w1, r1 = image_array.shape
        median_list = []
        median_value = 0
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    median_value = np.median(mesh[:,:,(rgb)])
                    median_list.append(median_value)
                    
        sharpen_mesh_array = np.array(median_list, dtype='uint8')
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        return sharpen_mesh_array_reshape
    
    def average_filter(image_array, n = 3):
        '''
        中值濾波
        x方向卷積
        n最小為3 (限奇數)
        '''
        h, w, r= image_array.shape
        zero_array = np.zeros(1)
        padding = int(n//2)
        for i in range(padding):
            image_array = np.insert(image_array, 0, zero_array, axis=0) #上
            image_array = np.insert(image_array, h, zero_array, axis=0) #下
            image_array = np.insert(image_array, 0, zero_array, axis=1) #左
            image_array = np.insert(image_array, w, zero_array, axis=1) #右
        
        h1, w1, r1 = image_array.shape
        average_list = []
        average_value = 0
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                if   (x+n > w1) or (y+n > h1):
                    continue
                else:
                    mesh = image_array[y:y+n, x:x+n]
                    #print(mesh.shape)
                    #print(y, x)
                for rgb in range(3):
                    average_value = np.mean(mesh[:,:,(rgb)])
                    average_list.append(average_value)
                    
        sharpen_mesh_array = np.array(average_list, dtype='uint8')
        sharpen_mesh_array_reshape = sharpen_mesh_array.reshape([h, w ,(r)])
        return sharpen_mesh_array_reshape
    
    def contrast_histogram_equalization_v(image_array_input):
        '''
        另一種寫法:
        import matplotlib.pyplot as plt
        import numpy as np
        import cv2


        img = cv2.imread('fff.jpg', 0)
        hist, bins = np.histogram(img.flatten(), 256, [0, 256])

        cdf = hist.cumsum()
        cdf_normalized = cdf * hist.max() / cdf.max()

        plt.plot(cdf_normalized, color='b')
        plt.hist(img.flatten(), 256, [0, 256], color = 'r')
        plt.xlim([0, 256])
        plt.legend(('cdf', 'histogram'), loc='upper left')
        plt.show()
        
        # calculate cdf
        cdf_m = np.ma.masked_equal(cdf, 0)
        cdf_m = (cdf_m - cdf_m.min())*255 / (cdf_m.max() - cdf_m.min())
        cdf = np.ma.filled(cdf_m, 0).astype('uint8')

        # mapping
        img2 = cdf[img]
        res = np.hstack((img, img2))
        plt.figure(figsize=(10,10))
        plt.imshow(res, cmap="gray")
        plt.show()
        
        #區間0~256 放入array中元素對應的個數
        hist, bins = np.histogram(img2.flatten(), bins= 256, range=[0, 256])

        cdf = hist.cumsum()
        cdf_normalized = cdf * hist.max() / cdf.max()

        plt.plot(cdf_normalized, color='b')
        plt.hist(img2.flatten(), 256, [0, 256], color = 'r')
        plt.xlim([0, 256])
        plt.ylim([0, 7000])
        plt.legend(('cdf', 'histogram'), loc='upper left')
        plt.show()
        '''
        image_array_input = other_class.RGB_to_HSV(image_array_input)
        
        image_array = copy.copy(image_array_input)
        h1, w1, r1 = image_array.shape
        v_list = [] # array中所有的 v值
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                v_list.append(image_array[y,x,(2)])
        # 因為 v_list為小數，為了方便建立0計數、桶子排派序法，故先轉成array *255再轉回list。
        v_list_array = np.array(v_list)
        v_list_array = v_list_array*255
        v_list_0_256 = v_list_array.tolist()
        v_list_0_256 = np.array(v_list_0_256, dtype='int')
        v_list_0_256 = v_list_0_256.tolist()

        #桶子排序法
        v_count = [] #存放強度值從0~255由小到大的count。(包含0計數)
        # 可能有數量為0的強度值  pdf
        for i in range(256):
            v_count.append(0)
        for v_value in v_list_0_256:
            v_count[v_value] = v_count[v_value] + 1

        v_value = [] # 0~255強度值
        for i in range(256):
            v_value.append(i)

        #v_count_cdf = np.cumsum(v_count) 同下列程式碼
        v_count_cdf = []
        for i in range(256):
            temp = 0
            for j in range(i+1):
                temp = temp + v_count[j]
            v_count_cdf.append(temp)
        v_count_cdf = np.array(v_count_cdf)

        MN = h1 * w1
        '''
        v_count_cdf      : v_count的cdf ，最後一項等於MN(h*w)
        max(v_count)     : 0~256中數量最多的值
        v_count_cdf.max(): 等於MN(h*w)
        '''
        # 這裡等化的動作是為了將兩種圖合在一起，所以特別針對數量(y)做了等化
        v_count_cdf_normalized = v_count_cdf * max(v_count) / v_count_cdf.max() #等化後 藍線最大值為 強度值的最大count


        # calculate cdf
        # (v_count_cdf[i] - v_count[0]) / (MN - v_count[0]) 要減掉 v_count[0]，不然會出事。
        L = 256
        scale = (L-1)
        a=[]
        cdf_list = []
        for i in range(256):
            cdf = scale * (v_count_cdf[i] - v_count[0]) / (MN - v_count[0])
            cdf_list.append(round(cdf))

        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                a = int(image_array[y, x, (2)] * 255)
                index = v_value.index(a, 0, len(v_value))
                image_array[y, x, (2)] = cdf_list[index] /255
                
        image_array = other_class.HSV_to_RGB(image_array)
        return image_array
    
    def contrast_histogram_equalization_rgb(image_array):
        '''
        這裡的HE方法並不能用在RGB圖片上，因為分別對R、G、B做了直方圖等化後，圖像會失真。
        所以，這個方法只能使用在R、G、B強度值相等的圖上。
        
        解決辦法 : 必須先將RGB轉成HSV，再對V做HE，再轉回RGB。
        '''
        import copy
        h1, w1, r1 = image_array.shape
        # 將array中的RGB分別存成list
        r_list = []
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                r_list.append(image_array[y, x,(0)])

        g_list = []
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                g_list.append(image_array[y, x,(1)])

        b_list = []
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                b_list.append(image_array[y,x,(2)])

        # 桶子排序法，會依據list內的灰階值計數。
        L = 256
        r_bucket_grey_value = [] #將圖片出現過的 強度值 儲存起來
        g_bucket_grey_value = []
        b_bucket_grey_value = []
        r_bucket = [] #每個桶子存取相對應grey value的個數(count)。
        g_bucket = []
        b_bucket = []
        for i in range(L):
            r_bucket.append(0)
        # 將grey_value一個一個從list中讀取，並且丟到相對應的桶子
        for grey_value in r_list:
            if not(grey_value in r_bucket_grey_value):
                r_bucket_grey_value.append(grey_value)
            r_bucket[grey_value] = r_bucket[grey_value] + 1

        for i in range(L):
            g_bucket.append(0)
        # 將grey_value一個一個從list中讀取，並且丟到相對應的桶子
        for grey_value in g_list:
            if not(grey_value in g_bucket_grey_value):
                g_bucket_grey_value.append(grey_value)
            g_bucket[grey_value] = g_bucket[grey_value] + 1 

        for i in range(L):
            b_bucket.append(0)
        # 將grey_value一個一個從list中讀取，並且丟到相對應的桶子
        for grey_value in b_list:
            if not(grey_value in b_bucket_grey_value):
                b_bucket_grey_value.append(grey_value)
            b_bucket[grey_value] = b_bucket[grey_value] + 1

        #另外儲存bucket，這裡的bucket內有強度值為0，則拿掉。目的是為了r_bucket_grey_value和r_bucket的數量相符，可以畫直方圖。
        r_bucket_nonzero = copy.copy(r_bucket)
        g_bucket_nonzero = copy.copy(g_bucket)
        b_bucket_nonzero = copy.copy(b_bucket)
        for i in range(r_bucket_nonzero.count(0)):
            r_bucket_nonzero.remove(0)
        for i in range(g_bucket_nonzero.count(0)):
            g_bucket_nonzero.remove(0)
        for i in range(b_bucket_nonzero.count(0)):
            b_bucket_nonzero.remove(0)

        # 將r_bucket_grey_value依據大小排列。r_bucket_grey_value和r_bucket_nonzero的數量是一樣的
        # r_bucket_grey_value : 紀錄bucket出現過的強度值
        # r_bucket_nonzero    : 將計數不為0的強度值數量記錄起來。而r_bucket包含0，所以長度一定是256
        r_bucket_grey_value = sorted(r_bucket_grey_value)
        g_bucket_grey_value = sorted(g_bucket_grey_value)
        b_bucket_grey_value = sorted(b_bucket_grey_value)

        a = []
        for i in range(L):
            a.append(i)
        r_bucket_grey_value_0_256 = a
        g_bucket_grey_value_0_256 = a
        b_bucket_grey_value_0_256 = a
        '''
        # 將bugket內的資料丟回r_list並且grey_value從小排到大。
        index = 0
        for i in range(len(bucket)):
            if bucket[i] != 0:
                for j in range(bucket[i]):
                    r_list[index] = i
                    index += 1
        '''
        '''
        # 建立字典
        r_bucket_grey_value = []
        for i in range(max_unit8):
            r_bucket_grey_value.append(i)

        r_hist_dict = dict(zip(r_bucket_grey_value, r_bucket))
        '''
        '''
        # 列印直方圖(histogram)
        import matplotlib.pylab as plt
        plt.plot(r_bucket_grey_value, r_bucket)
        plt.show()
        '''
        # 直方圖等化 histogram equalization
        r_range = L-1
        MN = h1*w1
        p = 0
        s_r = [] # 更新 強度值
        s_g = []
        s_b = []
        r_r = [] # 紀錄 r 對應 s，方便之後將這些 s取代原本的 強度值
        r_g = []
        r_b = []
        for k in range(r_range + 1):
            p_temp = 0
            count = 0
            if (k in r_bucket_grey_value_0_256):
                for j in range(k):
                    p_temp = p_temp + r_bucket[j]
                p = r_range * p_temp
                r_r.append(k)
            s_r.append(round(p / MN)) # 更新的強度值要四捨五入

        for k in range(r_range + 1):
            p_temp = 0
            if (k in g_bucket_grey_value_0_256):
                for j in range(k):
                    p_temp = p_temp + g_bucket[j]
                p = r_range * p_temp
                r_g.append(k)
            s_g.append(round(p / MN)) # 更新的強度值要四捨五入

        for k in range(r_range + 1):
            p_temp = 0
            if (k in b_bucket_grey_value_0_256):
                for j in range(k):
                    p_temp = p_temp + b_bucket[j]
                p = r_range * p_temp
                r_b.append(k)
            s_b.append(round(p / MN)) # 更新的強度值要四捨五入

        s_r_dict_r = dict(zip(r_r, s_r)) #{原強度值, 新強度值}
        s_r_dict_g = dict(zip(r_g, s_g))
        s_r_dict_b = dict(zip(r_b, s_b))

        # 將原本的array內的 強度值 更新。
        update_image_array = np.zeros([h1, w1 ,(r1)], dtype='uint8')
        for y in np.arange(0, h1, 1):
            for x in np.arange(0, w1, 1):
                update_image_array[y, x, (0)] = s_r_dict_r.get(image_array[y, x,(0)])
                update_image_array[y, x, (1)] = s_r_dict_g.get(image_array[y, x,(1)])
                update_image_array[y, x, (2)] = s_r_dict_b.get(image_array[y, x,(2)])
        return update_image_array
    
    def histogram_equalization_HSV_matching(image_array_input, cdf_match):
        image_array_input = other_class.RGB_to_HSV(image_array_input)
        image_array = copy.copy(image_array_input)

        '''
        #值方圖匹配(指定)
        輸入想匹配用的cdf曲線(RGB array)，並對HSV的V(明度)做直方圖匹配
        比較原圖、HE後圖、匹配後的圖 三者的直方圖和圖片比較
        y_array_match為 指定的cdf
        '''
        # 只取v來做HE。  v = 0~1，為了方便計算，都*255變成0~255。
        image_array_v = image_array[:,:,(2)]*255
        image_array_v = image_array_v.astype('uint8')
        # 對 v做hist
        hist_r,bins = np.histogram(image_array_v.flatten(), bins=256, range=[0,256])

        #對r做 pdf累積=cdf
        cdf_r = hist_r.cumsum()
        # cdf是數量的累積，正規化到最大值為其中中數量最大的值，這樣cdf曲線可以和hist放在同一張圖。
        cdf_r_normalized = cdf_r * hist_r.max() / cdf_r.max()
        
        plt.figure(figsize=(20,20))
        plt.subplot(421)
        plt.title('initial image')
        plt.plot(np.arange(0,256,1) ,cdf_r_normalized, color='b')
        plt.hist(image_array_v.flatten(), bins=256, range=[0,256], color = 'c')
        plt.xlim([0, 256])
        plt.legend(('cdf', 'histogram'), loc='upper left')

        # calculate cdf
        # np.ma.masked_equal(array, 0)會設置一個遮罩，可以將array中為0的不會被做任何運算
        cdf_m = np.ma.masked_equal(cdf_r, 0)
        cdf_m = (cdf_m - cdf_m.min())*255 / (cdf_m.max() - cdf_m.min())
        # np.ma.filled(array, 0) 將為True的遮罩放入0，等於是之前拿掉0 現在只是重新放回來。
        cdf = np.ma.filled(cdf_m, 0).astype('uint8')

        # mapping映射 ，cdf四捨五入後為0~255的HE排列。 
        # cdf[array]會將array的值變成'索引值'帶入cdf，根據cdf索引值位置的值來做替換
        '''ex:
        a = np.array([0,1,2,3,4,4,6])
        b = np.array([5,5,5])
        a[b]
        >>>array([4, 4, 4])
        '''
        image_array_v_he = cdf[image_array_v]

        # HE完成後，跟之前一樣出一張cdf和pdf的圖
        # 區間0~256 放入array中元素對應的個數
        hist_s, bins_s = np.histogram(image_array_v_he.flatten(), bins= 256, range=[0, 256])
        cdf_s = hist_s.cumsum()
        cdf_s_normalized = cdf_s * hist_s.max() / cdf_s.max()

        plt.subplot(422)
        plt.imshow(other_class.HSV_to_RGB(image_array_input))

        plt.subplot(423)
        plt.title('histogram equalization')
        plt.plot(np.arange(0,256,1), cdf_s_normalized, color='g')
        plt.hist(image_array_v_he.flatten(), 256, [0, 256], color = 'c')
        plt.xlim([0, 256])
        plt.legend(('cdf', 'histogram'), loc='upper left')
        #print(cdf_s_normalized)

        plt.subplot(424)
        image_array[:,:,(2)] = image_array_v_he/255
        plt.imshow(other_class.HSV_to_RGB(image_array))
        #----------------------------------------------------
        
        #cdf_r_S = np.around(255 * cdf_s / cdf_s.max())
        cdf_s_G = np.around(255 * cdf_match / cdf_match.max())

        z = []
        for i in range(256):
            diff = abs(cdf_s_G - cdf[i])
            diff = diff.tolist()
            index_diff = diff.index(min(diff))
            z.append(index_diff)
        z = np.array(z)
        image_array_v_he_matching = z[image_array_v]

        hist_z, bins_z = np.histogram(image_array_v_he_matching.flatten(), bins= 256, range=[0, 256])
        cdf_z = hist_z.cumsum()
        cdf_z_normalized = cdf_z * hist_z.max() / cdf_z.max()

        plt.subplot(425)
        plt.title('histogram matching')
        plt.plot(np.arange(0,256,1), cdf_z_normalized, color='r')
        plt.hist(image_array_v_he_matching.flatten(), 256, [0, 256], color = 'c')
        plt.xlim([0, 256])
        plt.legend(('cdf', 'histogram'), loc='upper left')

        plt.subplot(426)
        image_array[:,:,(2)] = image_array_v_he_matching/255
        plt.imshow(other_class.HSV_to_RGB(image_array))
        
        plt.subplot(427)
        plt.plot(np.arange(0,256,1) ,cdf_r_normalized, color='b')
        plt.plot(np.arange(0,256,1), cdf_s_normalized, color='g')
        plt.plot(np.arange(0,256,1), cdf_z_normalized, color='r')
        plt.legend(('initial image', 'histogram equalization','histogram matching'), loc='upper left')
        plt.show()
        
        
        image_array = other_class.HSV_to_RGB(image_array)
        return image_array
    
    def histogram_equalization_partial(image_array_input, n=3, E=4, k0=0.4, k1=0.02, k2=0.4):
        '''
        # 直方圖統計增強影像
        輸入為RGB array
        k0 : 值小於0的常數。
        k1 < k2，限制variance_grobal的上下限。
        k2 > 1對增強亮的區域有興趣
        k2 < 1對暗的區域有興趣
        '''
        image_array_input = other_class.RGB_to_HSV(image_array_input)
        image_array = copy.copy(image_array_input)
        image_array_v = image_array[:,:,(2)]

        h, w = image_array_v.shape
        zero_array = np.zeros(1)
        padding = int(n//2)
        padding_array_h = image_array_v[0,:]
        for i in range(padding):
            image_array_v = np.insert(image_array_v, 0, padding_array_h, axis=0) #上
            image_array_v = np.insert(image_array_v, h, padding_array_h, axis=0) #下
        padding_array_w = image_array_v[:,0]
        for i in range(padding):
            image_array_v = np.insert(image_array_v, 0, padding_array_w, axis=1) #左
            image_array_v = np.insert(image_array_v, w, padding_array_w, axis=1) #右
        h1, w1 = image_array_v.shape

        m_sy        = 0 # sliding_window中 強度值 的 平均強度
        variance_sy = 0

        m_grobal        = np.sum(image_array_v)/(h1*w1)
        variance_grobal = (np.sum((image_array_v - m_grobal)**2)/(h1*w1))**(0.5)
        print('m_grobal : ',m_grobal)
        print('variance_grobal : ',variance_grobal)

        a = []
        count = 0
        for y in range(h):
            for x in range(w):
                sliding_window = image_array_v[y:y+n, x:x+n]
                m_sy = np.sum(sliding_window)/(n*n)
                variance_sy = (np.sum((sliding_window - m_sy)**2)/(n*n))**(0.5)

                if (m_sy <= (k0*m_grobal)) & ((k1*variance_grobal) <= variance_sy <= (k2*variance_grobal)):
                    g = E * image_array_v[y, x]
                    if g > 1:
                        g = 1
                    elif g < 0:
                        g = 0
                    else:
                        pass
                    a.append(g)
                else:
                    count += 1
                    a.append(image_array_v[y, x])

        new_image_array_v = np.array(a)
        new_image_array_v = new_image_array_v.reshape(h, w)
        image_array[:,:,(2)] = new_image_array_v
        image_array = other_class.HSV_to_RGB(image_array)
        return image_array
    
    
    
    def binarization(image_array, threshold_low=50, threshold_high=205):
        h, w, r = image_array.shape
        
        new_image_array = copy.copy(image_array)
        for y in np.arange(0, h, 1):
            for x in np.arange(0, w, 1):
                for rgb in range(r):
                    if (new_image_array[y, x,(rgb)] <= threshold_low ):
                        new_image_array[y, x,(rgb)] = 0
                    elif (new_image_array[y, x,(rgb)] >= threshold_high ):
                        new_image_array[y, x,(rgb)] = 255
                    else:
                        pass
        return new_image_array
    
    def gamma_correction_HSV(image_array_input, gamma = 0.67, c = 1):
        '''
        gamma和亮化暗化不一樣，對於接近極值的強度值影響越小，故不會損失圖片的細節。(調亮超過255的部分也是255，細節會不見)
        輸入需為HSV
        '''
        image_array_input = other_class.RGB_to_HSV(image_array_input)
        image_array = copy.copy(image_array_input)
        
        # gamma = r時為恆等值轉換
        h, w, r = image_array.shape
        
        # 雖然也可以用for來達到跟flatten()的效果，但比較麻煩且之前已經做過，故直接飲用函式庫
        b = image_array[:,:,(2)].flatten()

        # gamma公式  s=cr**gamma  其中r必須正規化
        b_output = c*b**gamma
        
        '''r_output = r_output.tolist()
        g_output = g_output.tolist()
        b_output = b_output.tolist()'''
        # 把轉換玩的s放回去image_array
        count = 0
        for y in range(h):
            for x in range(w):
                image_array[y, x, (2)] = b_output[count]
                count = count +1
        image_array = other_class.HSV_to_RGB(image_array)
        return image_array
    
    def gamma_correction_RGB(image_array_input, gamma = 0.67, c = 1):
        '''
        gamma和亮化暗化不一樣，對於接近極值的強度值影響越小，故不會損失圖片的細節。(調亮超過255的部分也是255，細節會不見)
        輸入為RGB
        '''
        image_array = copy.copy(image_array_input)
        
        # gamma = r時為恆等值轉換
        h, w, r = image_array.shape
        # 雖然也可以用for來達到跟flatten()的效果，但比較麻煩且之前已經做過，故直接飲用函式庫
        # r,g,b正規化
        r = image_array[:,:,(0)].flatten()/255
        g = image_array[:,:,(1)].flatten()/255
        b = image_array[:,:,(2)].flatten()/255

        # gamma公式  s=cr**gamma
        r_output = (c*r**gamma)*255
        g_output = (c*g**gamma)*255
        b_output = (c*b**gamma)*255

        '''r_output = r_output.tolist()
        g_output = g_output.tolist()
        b_output = b_output.tolist()'''
        # 把轉換玩的s放回去image_array
        count = 0
        for y in range(h):
            for x in range(w):
                image_array[y, x, (0)] = r_output[count]
                image_array[y, x, (1)] = g_output[count]
                image_array[y, x, (2)] = b_output[count]
                count = count +1
        return image_array
                    